In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import os
import gc

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score

from tqdm import tqdm

# model
import lightgbm as lgb

SEED=42
LABEL=None

In [0]:
path = '/content/drive/My Drive/bigcontest2019/'
os.chdir(path)

In [0]:
train_activity = pd.read_csv('data/train/train_activity.csv')
# train_combat = pd.read_csv('data/train/train_combat.csv')
train_payment = pd.read_csv('data/train/train_payment.csv')
# train_pledge = pd.read_csv('data/train/train_pledge.csv')
# train_trade = pd.read_csv('data/train/train_trade.csv')

test1_activity = pd.read_csv('data/test/test1_activity.csv')
# test1_combat = pd.read_csv('data/test/test1_combat.csv')
test1_payment = pd.read_csv('data/test/test1_payment.csv')
# test1_pledge = pd.read_csv('data/test/test1_pledge.csv')
# test1_trade = pd.read_csv('data/test/test1_trade.csv')

test2_activity = pd.read_csv(path + 'data/test/test2_activity.csv')
# test2_combat = pd.read_csv(path + 'data/test/test2_combat.csv')
test2_payment = pd.read_csv(path + 'data/test/test2_payment.csv')
# test2_pledge = pd.read_csv(path + 'data/test/test2_pledge.csv')
# test2_trade = pd.read_csv(path + 'data/test/test2_trade.csv')

train_label = pd.read_csv('data/train/train_label.csv')
# train_label['total_spent'] = train_label['survival_time']*train_label['amount_spent']
# train_label['adjust_survival_time'] = train_label['survival_time']
# train_label.loc[train_label['adjust_survival_time']<64, 'adjust_survival_time'] = 1
# train_label.loc[train_label['adjust_survival_time']==64, 'adjust_survival_time'] = 0
# train_label = pd.merge(train_activity[['acc_id', 'char_id']], train_label, how='left', on='acc_id')

In [0]:
def feature_extraction(data, feature, method, week):
    '''
    data - DataFrame
    feature - column name
    metod - ex) sum, unique, min, max ...etc
    '''
    if week==1:
        data = data[(data['day']>0) & (data['day']<8)]
    elif week==2:
        data = data[(data['day']>7) & (data['day']<15)]
    elif week==3:
        data = data[(data['day']>14) & (data['day']<22)]
    else:
        data = data[(data['day']>21) & (data['day']<29)]
        
    train = pd.DataFrame(np.unique(data['acc_id']), columns=['acc_id'])
    
    standard_feature = 'acc_id'
    if method=='nunique':
        data = data.groupby(standard_feature)[feature].nunique()
    elif method=='sum':
        data = data.groupby(standard_feature)[feature].sum()
    elif method=='min':
        data = data.groupby(standard_feature)[feature].min()
    elif method=='max':
        data = data.groupby(standard_feature)[feature].max()
    elif method=='mean':
        data = data.groupby(standard_feature)[feature].mean()
    elif method=='median':
        data = data.groupby(standard_feature)[feature].median()
    elif method=='skew':
        data = data.groupby(standard_feature)[feature].skew()
    elif method=='kurt':
        data = data.groupby(standard_feature)[feature].apply(pd.DataFrame.kurt)
    elif method=='count':
        data = data.groupby(standard_feature)[feature].count()
    elif method=='std':
        data = data.groupby(standard_feature)[feature].std()
    else:
        return print('method is no return')
    
    data = data.reset_index().rename(columns={feature:feature+'_'+method})
    train = pd.merge(train, data, how='left', on='acc_id')
    
    return train

In [0]:
def activity_proprocess(data, feature, method, week=1):
    '''
    data - activity dataframe
    week - week ex)1, 2, 3, 4
    '''
    if week==1:
        data = data[(data['day']>0) & (data['day']<8)]
    elif week==2:
        data = data[(data['day']>7) & (data['day']<15)]
    elif week==3:
        data = data[(data['day']>14) & (data['day']<22)]
    else:
        data = data[(data['day']>21) & (data['day']<29)]
    
    train = feature_extraction(data, feature, method, week)
    
    return train

# train

## def function

In [0]:
def activity_merge(train_week, data, week=1, on='acc_id'):
    WEEK=week
    ON=on
    for method in ['nunique', 'count']:
        train_week = pd.merge(train_week, activity_proprocess(DATA, feature='server', method=method, week=WEEK), how='left', on=ON)

    for feature in ['playtime', 'npc_kill', 'solo_exp', 'party_exp', 'quest_exp', 'rich_monster', 'death', 'revive', 'exp_recovery', 'fishing', 'private_shop', 'game_money_change', 'enchant_count']:
        for method in ['nunique', 'max', 'min', 'mean', 'median', 'skew', 'kurt', 'sum', 'std']:
            train_week = pd.merge(train_week, activity_proprocess(DATA, feature=feature, method=method, week=WEEK), how='left', on=ON)
    
    return train_week

In [0]:
def payment_merge(train_week, data, week=1, on='acc_id'):
    WEEK=week
    ON=on
    feature = 'amount_spent'
    
    for method in ['nunique', 'max', 'min', 'mean', 'median', 'skew', 'kurt', 'sum', 'std']:
        train_week = pd.merge(train_week, activity_proprocess(DATA, feature=feature, method=method, week=WEEK), how='left', on=ON)
    
    return train_week

## activity

In [0]:
DATA = train_activity
ON = 'acc_id'

train_week1 = activity_proprocess(DATA, feature='char_id', method='nunique', week=1)
train_week2 = activity_proprocess(DATA, feature='char_id', method='nunique', week=2)
train_week3 = activity_proprocess(DATA, feature='char_id', method='nunique', week=3)
train_week4 = activity_proprocess(DATA, feature='char_id', method='nunique', week=4)

train_week1 = activity_merge(train_week1, DATA, week=1)
print('train_week1 complete')
train_week2 = activity_merge(train_week2, DATA, week=2)
print('train_week2 complete')
train_week3 = activity_merge(train_week3, DATA, week=3)
print('train_week3 complete')
train_week4 = activity_merge(train_week4, DATA, week=4)
print('train_week4 complete')

## Payment

In [0]:
DATA = train_payment
ON = 'acc_id'

train_week1 = payment_merge(train_week1, DATA, week=1)
print('train_week1 complete')
train_week2 = payment_merge(train_week2, DATA, week=2)
print('train_week2 complete')
train_week3 = payment_merge(train_week3, DATA, week=3)
print('train_week3 complete')
train_week4 = payment_merge(train_week4, DATA, week=4)
print('train_week4 complete')

train_week1 complete
train_week2 complete
train_week3 complete
train_week4 complete


In [0]:
# survival time

train_week1 = pd.merge(train_week1, train_label[['acc_id', 'survival_time']], how='left', on='acc_id')
train_week2 = pd.merge(train_week2, train_label[['acc_id', 'survival_time']], how='left', on='acc_id')
train_week3 = pd.merge(train_week3, train_label[['acc_id', 'survival_time']], how='left', on='acc_id')
train_week4 = pd.merge(train_week4, train_label[['acc_id', 'survival_time']], how='left', on='acc_id')

train_week1['survival_time'] = train_week1['survival_time']+21
train_week1['survival_time'] = train_week1['survival_time']+14
train_week1['survival_time'] = train_week1['survival_time']+7

train_week1.loc[train_week1['survival_time']>64, 'survival_time'] = 64
train_week2.loc[train_week2['survival_time']>64, 'survival_time'] = 64
train_week3.loc[train_week3['survival_time']>64, 'survival_time'] = 64

train_week = pd.concat([train_week1, train_week2, train_week3, train_week4]).reset_index(drop=True)

In [0]:
# amount spent
train_week = pd.merge(train_week, train_label[['acc_id', 'amount_spent']], how='left', on='acc_id')

# test

In [0]:
DATA = test1_activity
ON = 'acc_id'

test1_week1 = activity_proprocess(DATA, feature='char_id', method='nunique', week=1)
test1_week2 = activity_proprocess(DATA, feature='char_id', method='nunique', week=2)
test1_week3 = activity_proprocess(DATA, feature='char_id', method='nunique', week=3)
test1_week4 = activity_proprocess(DATA, feature='char_id', method='nunique', week=4)

test1_week1 = activity_merge(test1_week1, DATA, week=1)
test1_week2 = activity_merge(test1_week2, DATA, week=2)
test1_week3 = activity_merge(test1_week3, DATA, week=3)
test1_week4 = activity_merge(test1_week4, DATA, week=4)

DATA = test1_payment
test1_week1 = payment_merge(test1_week1, DATA, week=1)
test1_week2 = payment_merge(test1_week2, DATA, week=2)
test1_week3 = payment_merge(test1_week3, DATA, week=3)
test1_week4 = payment_merge(test1_week4, DATA, week=4)

In [0]:
test1_week = pd.concat([test1_week1, test1_week2, test1_week3, test1_week4]).reset_index(drop=True)

In [0]:
DATA = test2_activity
ON = 'acc_id'

test2_week = activity_proprocess(DATA, feature='char_id', method='nunique', week=1)
test2_week = activity_proprocess(DATA, feature='char_id', method='nunique', week=2)
test2_week = activity_proprocess(DATA, feature='char_id', method='nunique', week=3)
test2_week = activity_proprocess(DATA, feature='char_id', method='nunique', week=4)

test2_week1 = activity_merge(test2_week, DATA, week=1)
test2_week2 = activity_merge(test2_week, DATA, week=2)
test2_week3 = activity_merge(test2_week, DATA, week=3)
test2_week4 = activity_merge(test2_week, DATA, week=4)

DATA = test2_payment
test2_week1 = payment_merge(test2_week, DATA, week=1)
test2_week2 = payment_merge(test2_week, DATA, week=2)
test2_week3 = payment_merge(test2_week, DATA, week=3)
test2_week4 = payment_merge(test2_week, DATA, week=4)

# model

## survival_time

In [0]:
seed=42
LABEL='survival_time'
train_df, valid_df = train_test_split(train_week.fillna(0), test_size=0.2, random_state=seed, shuffle=True, stratify=train_week[LABEL])

In [0]:
feature_ = train_week1.drop(columns=['acc_id', LABEL]).columns
params = {
    'objective':'multiclass',
    'num_class':64,
    "boosting": "gbdt",
    'learning_rate': 0.03,
    'subsample' : 0.6,
    'sumsample_freq':1,
    'colsample_bytree':0.221856,
    'max_depth': 16,
    'max_bin':255,
    "lambda_l1": 0.25,
    "lambda_l2": 1,
    'min_child_weight': 0.2,
    'min_child_samples': 20,
    'min_gain_to_split':0.02,
    'min_data_in_bin':3,
    'bin_construct_sample_cnt':5000,
    'cat_l2':10,
    'verbose':-1,
    'nthread':-1,
    'seed':seed
}

trn_label = train_df[LABEL] - 1
val_label = valid_df[LABEL] - 1

ttt = lgb.Dataset(train_df[feature_], label=trn_label)
vvv = lgb.Dataset(valid_df[feature_], label=val_label)

lgb_model = lgb.train(params, ttt, 5000, valid_sets = [ttt, vvv], early_stopping_rounds = 50, verbose_eval=100)

# oof = lgb_model.predict(valid_df[feature_])
# oof = np.argmax(oof, axis=1)+1

preds = lgb_model.predict(test1_week[feature_].fillna(0))
preds = np.argmax(preds, axis=1)+1

# preds2 = lgb_model.predict(test2_week[feature_].fillna(0))
# preds2 = np.argmax(preds2, axis=1)+1

Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 1.48572	valid_1's multi_logloss: 1.73791
[200]	training's multi_logloss: 1.26735	valid_1's multi_logloss: 1.69032
[300]	training's multi_logloss: 1.11433	valid_1's multi_logloss: 1.67299
[400]	training's multi_logloss: 0.993272	valid_1's multi_logloss: 1.66399
[500]	training's multi_logloss: 0.892156	valid_1's multi_logloss: 1.65863
[600]	training's multi_logloss: 0.805847	valid_1's multi_logloss: 1.65606
[700]	training's multi_logloss: 0.731002	valid_1's multi_logloss: 1.65528
Early stopping, best iteration is:
[689]	training's multi_logloss: 0.738795	valid_1's multi_logloss: 1.65514


## amount spent

In [0]:
seed=42
LABEL='amount_spent'
train_df, valid_df = train_test_split(train_week.fillna(0), test_size=0.2, random_state=seed, shuffle=True)

In [0]:
feature_ = train_week.drop(columns=['acc_id', 'survival_time', LABEL]).columns
params = {
    'objective':'regression',
    "boosting": "gbdt",
    'learning_rate': 0.03,
    'subsample' : 0.6,
    'sumsample_freq':1,
    'colsample_bytree':0.221856,
    'max_depth': 16,
    'max_bin':255,
    "lambda_l1": 0.25,
    "lambda_l2": 1,
    'min_child_weight': 0.2,
    'min_child_samples': 20,
    'min_gain_to_split':0.02,
    'min_data_in_bin':3,
    'bin_construct_sample_cnt':5000,
    'cat_l2':10,
    'verbose':-1,
    'nthread':-1,
    'metrics':'mse',
    'seed':seed
}

trn_label = train_df[LABEL]
val_label = valid_df[LABEL]

ttt = lgb.Dataset(train_df[feature_], label=trn_label)
vvv = lgb.Dataset(valid_df[feature_], label=val_label)

lgb_model = lgb.train(params, ttt, 5000, valid_sets = [ttt, vvv], early_stopping_rounds = 50, verbose_eval=100)
preds_spent = lgb_model.predict(test1_week[feature_].fillna(0))
# preds2_spent = lgb_model.predict(test2_week[feature_].fillna(0))

Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.33375	valid_1's l2: 0.385441
Early stopping, best iteration is:
[97]	training's l2: 0.334721	valid_1's l2: 0.385333


In [0]:
ppp = pd.concat([test1_week['acc_id'], pd.DataFrame(preds_spent, columns=['amount_spent'])], 1).groupby('acc_id').max().reset_index()

In [0]:
pp = pd.concat([test1_week['acc_id'], pd.DataFrame(preds, columns=['survival_time'])], 1).groupby('acc_id').mean().reset_index()

In [0]:
pp = pd.merge(pp.astype(int), ppp, how='left', on='acc_id')

In [0]:
pp['amount_spent'] = pp['amount_spent']*20

In [0]:
pp.to_csv('/content/test1_predict.csv', index=False)

# submission

In [0]:
test1_pred = pd.concat([test1_week['acc_id'], pd.DataFrame(preds, columns=['survival_time']), pd.DataFrame(preds_spent*25, columns=['amount_spent'])], 1)
test1_pred.to_csv('/content/test1_predict.csv', index=False)

In [0]:
test2_pred = pd.concat([test2_week['acc_id'], pd.DataFrame(preds2, columns=['survival_time']), pd.DataFrame(preds2_spent*10, columns=['amount_spent'])], 1)
test2_pred.to_csv('/content/test2_predict.csv', index=False)